# Training an XOR network using PySyft

In this notebook we'll be training an XOR network using PySyft. 
What's special about this is that the training data will be divided between two domain nodes that are owned by different parties.

In [1]:
import syft as sy
import numpy as np
sy.logger.remove()

/home/azureuser/.local/share/virtualenvs/python39-2vM0bQN-/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Logging into the domain Nodes

In [2]:
# We will login into Canada and Italy domain node
om = sy.login(email="sheldon@caltech.edu", password="bazinga", port=8081)
fb = sy.login(email="sheldon@caltech.edu", password="bazinga", port=8082)
# om = sy.login(email="info@openmined.org", password="changethis", port=8081)
# fb = sy.login(email="info@openmined.org", password="changethis", port=8082)

Connecting to localhost... done! 	 Logging into openmined... done!
Connecting to localhost... done! 	 Logging into facebook... done!


In [3]:
om.datasets

Idx,Name,Description,Assets,Id
[0],Our training data for XOR networks!,Collected on Jan 27 2022,"[""training_data""] -> int64[""training_targets""] -> int64",43e1955c-4a9b-4c5e-9130-0bc6773ae902


In [4]:
fb.datasets

Idx,Name,Description,Assets,Id
[0],Our training data for XOR networks!,Collected on Jan 27 2022,"[""training_data""] -> int64[""training_targets""] -> int64",9e999738-470a-4d1d-bcf8-c6292616a1f3


In [5]:
om.privacy_budget

9999999.0

In [6]:
fb.privacy_budget

9999999.0

In [7]:
om_train_data = om.datasets[-1]["training_data"]
om_targets_data = om.datasets[-1]["training_targets"]
fb_train_data = fb.datasets[-1]["training_data"]
fb_targets_data = fb.datasets[-1]["training_targets"]

In [8]:

train_data = om_train_data.concatenate(fb_train_data)
targets_data = om_targets_data.concatenate(fb_targets_data)
X = train_data
y = targets_data

kj/filesystem-disk-unix.c++:1690: warning: PWD environment variable doesn't match current directory; pwd = /home/azureuser


In [9]:
def relu(x,deriv=False):
    if deriv==True:
        return x>0
    return x*(x>0)

In [10]:
layer0_weights = 2*np.random.random((3,4)) - 1
layer1_weights = 2*np.random.random((4,1)) - 1

In [11]:
for j in range(1):
    # Forward propagation
    layer1_inputs = relu(X @ layer0_weights)  ; layer1_inputs.block
    layer2_inputs = relu(layer1_inputs @ layer1_weights) ; layer2_inputs.block 
    
    # Calculate errors
    layer2_inputs_delta = (y - layer2_inputs)* relu(layer2_inputs,deriv=True) ; layer2_inputs_delta.block
    layer1_inputs_delta = (layer2_inputs_delta@(layer1_weights.T)) * relu(layer1_inputs,deriv=True) ; layer1_inputs_delta.block
    
    # Update weights
    layer1_weights  = layer1_weights + layer1_inputs.T @ layer2_inputs_delta ; layer1_weights.block
    layer0_weights =  layer0_weights + X.T @ layer1_inputs_delta ; layer0_weights.block

Computing...:   0%|          | 0/64 [00:00<?, ?it/s]/home/azureuser/PySyft/packages/syft/src/syft/core/tensor/smpc/utils.py:72: RuntimeWarning: overflow encountered in multiply
  res = op(np.empty(x_shape), np.empty(y_shape)).shape
Computing...: 100%|██████████| 64/64 [00:37<00:00,  1.68it/s]
/home/azureuser/PySyft/packages/syft/src/syft/core/tensor/smpc/utils.py:72: RuntimeWarning: overflow encountered in matmul
  res = op(np.empty(x_shape), np.empty(y_shape)).shape
Computing...: 100%|██████████| 64/64 [00:38<00:00,  1.66it/s]


In [23]:
layer0_weights_dp = layer0_weights.publish(sigma=1e4)
layer1_weights_dp = layer1_weights.publish(sigma=1e4)

In [24]:
print(layer0_weights_dp.get_copy())
print(layer1_weights_dp.get_copy())

[[[ 6.35980658e+07  2.07598704e+09 -1.44185272e+09  2.66647517e+09]
  [-3.88470379e+08 -1.77254397e+09 -3.14791919e+09 -5.68840714e+08]
  [ 1.66087688e+08 -2.80472044e+08  3.15939302e+09 -7.16520031e+08]]]
[[[-3.81614137e+09]
  [-1.61770617e+09]
  [-1.50810469e+05]
  [ 1.90622190e+09]]]


<hr>

<hr>
<hr>

Let's see how our privacy budget changed as a result of training for a single epoch:

In [ ]:
ds_domain1.privacy_budget

In [ ]:
ds_domain2.privacy_budget

And voila! We've trained a neural network using PySyft's adversaril differential privacy system and its secure multiparty computation system working in tandem.


<hr>
<hr>
Demo ends above- the cells below are for temporary Debugging:

In [ ]:
X

In [ ]:
layer0_weights

In [ ]:
layer1_inputs = relu(X @ layer0_weights)  ; layer1_inputs.block

In [ ]:
layer2_inputs = relu(layer1_inputs @ layer1_weights) ; layer2_inputs.block 

In [ ]:
layer2_inputs_delta = (y - layer2_inputs)* relu(layer2_inputs,deriv=True) ; layer2_inputs_delta.block

In [ ]:
layer1_inputs_delta = (layer2_inputs_delta@(layer1_weights.T)) * relu(layer1_inputs,deriv=True) ; layer1_inputs_delta.block

In [ ]:
layer1_weights  = layer1_inputs.T @ layer2_inputs_delta  + layer1_weights; layer1_weights.block

In [ ]:
layer0_weights =  X.T @ layer1_inputs_delta  + layer0_weights; layer0_weights.block

In [ ]:
res = X.T @ layer1_inputs_delta

In [ ]:
v1 = X.T
v2 = layer1_inputs_delta

In [ ]:
def get_val(val):
    t1 = val.child[0].get_copy()
    t2 = val.child[0].get_copy()
    return t1+t2

In [ ]:
v1 = get_val(v1)
v2 = get_val(v2)

In [ ]:
v1.child.min_vals.to_numpy() @ v2.child.min_vals.to_numpy()

In [ ]:
v2.child.min_vals

In [ ]:
layer0_weights_dp = layer0_weights.publish(sigma=1e4)
layer1_weights_dp = layer1_weights.publish(sigma=1e4)